# Data Cleaning & Preprocessing - Versione Integrata

**Approccio unificato che combina:**
- ✅ **Tuo progetto**: Feature engineering avanzate da purchase orders, gestione batch_id
- ✅ **Repository riferimento**: Aggregazione giornaliera (date_arrival, rm_id), Quantile Loss preparation

**Pipeline completa**: Raw data → Cleaning → Feature engineering → Daily aggregation → Output per ML

In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("🚀 Data Cleaning & Preprocessing - Versione Integrata")
print("   Combinando il meglio del tuo progetto + repository riferimento")

🚀 Data Cleaning & Preprocessing - Versione Integrata
   Combinando il meglio del tuo progetto + repository riferimento


In [13]:
# Carica receivals
receivals = pd.read_csv('../../data/kernel/receivals.csv')
print(f"📦 Receivals dataset: {receivals.shape}")
print(f"   Colonne: {receivals.columns.tolist()}")

# Carica purchase_orders
purchase_orders = pd.read_csv('../../data/kernel/purchase_orders.csv')
print(f"\n📋 Purchase Orders dataset: {purchase_orders.shape}")
print(f"   Colonne: {purchase_orders.columns.tolist()}")

📦 Receivals dataset: (122590, 10)
   Colonne: ['rm_id', 'product_id', 'purchase_order_id', 'purchase_order_item_no', 'receival_item_no', 'batch_id', 'date_arrival', 'receival_status', 'net_weight', 'supplier_id']

📋 Purchase Orders dataset: (33171, 12)
   Colonne: ['purchase_order_id', 'purchase_order_item_no', 'quantity', 'delivery_date', 'product_id', 'product_version', 'created_date_time', 'modified_date_time', 'unit_id', 'unit', 'status_id', 'status']


## 1. Load Data & Initial Analysis

In [14]:
print(f"\n🔍 Analisi stati di ricezione:")
print(receivals['receival_status'].value_counts())
print(f"\n📊 Distribuzione percentuale:")
print(receivals['receival_status'].value_counts(normalize=True) * 100)

# Mantieni TUTTI i record
# Nota: Non serve .copy() qui perché il merge subito dopo crea un nuovo DataFrame
data_clean = receivals
print(f"\n✅ Mantenuti TUTTI i record: {len(data_clean)} record")


🔍 Analisi stati di ricezione:
receival_status
Completed             122448
Finished unloading       106
Start unloading           32
Planned                    4
Name: count, dtype: int64

📊 Distribuzione percentuale:
receival_status
Completed             99.884167
Finished unloading     0.086467
Start unloading        0.026103
Planned                0.003263
Name: proportion, dtype: float64

✅ Mantenuti TUTTI i record: 122590 record


## 2. Merge con Purchase Orders

In [15]:
# NOTE: the two datasets have different time formats, so we need to standardize them before merging

# ✅ IMPORTANTE: Facciamo una copia per non modificare gli originali
# (utile se vogliamo riutilizzare i dataset originali per debug/confronti)

# receivals.csv
data_clean = data_clean.copy()  # Copia PRIMA di modificare in-place
data_clean['date_arrival'] = pd.to_datetime(
    data_clean['date_arrival'], utc=True
)

# purchase_orders.csv
purchase_orders = purchase_orders.copy()  # Copia PRIMA di modificare
for col in ['delivery_date', 'created_date_time', 'modified_date_time']:
    purchase_orders[col] = pd.to_datetime(
        purchase_orders[col], utc=True
    )

# Merge
print("\n🔗 Merge receivals con purchase_orders...")
data_clean = data_clean.merge(
    purchase_orders[['purchase_order_id', 'purchase_order_item_no', 'quantity',
                     'delivery_date', 'created_date_time', 'status', 'product_version']],
    on=['purchase_order_id', 'purchase_order_item_no'],
    how='left',
    suffixes=('', '_po')
)

print(f"✅ Merge completato: {data_clean.shape}")
print(f"\n📊 Record con match in purchase_orders: {data_clean['quantity'].notna().sum()} "
      f"({data_clean['quantity'].notna().sum()/len(data_clean)*100:.1f}%)")
print(f"📊 Record senza match: {data_clean['quantity'].isna().sum()} "
      f"({data_clean['quantity'].isna().sum()/len(data_clean)*100:.1f}%)")



🔗 Merge receivals con purchase_orders...
✅ Merge completato: (122590, 15)

📊 Record con match in purchase_orders: 122537 (100.0%)
📊 Record senza match: 53 (0.0%)


## 3. Target Variable Cleaning

In [16]:
print(f"\n📊 Analisi net_weight (TARGET variable):")
print(f"   Record totali: {len(data_clean)}")
print(f"   Record con net_weight mancante: {data_clean['net_weight'].isna().sum()}")
print(f"   Record con net_weight = 0: {(data_clean['net_weight'] == 0).sum()}")
print(f"   Record con net_weight < 0: {(data_clean['net_weight'] < 0).sum()}")

# Rimuovi solo record con peso mancante o <= 0
# NECESSARIO: net_weight è il target, deve essere valido per il training
initial_count = len(data_clean)
data_clean = data_clean[data_clean['net_weight'].notna() & (data_clean['net_weight'] > 0)]
print(f"\n✅ Dopo pulizia net_weight: {len(data_clean)} record (rimossi {initial_count - len(data_clean)})")
print(f"   Percentuale rimossa: {(initial_count - len(data_clean))/initial_count*100:.1f}%")


📊 Analisi net_weight (TARGET variable):
   Record totali: 122590
   Record con net_weight mancante: 68
   Record con net_weight = 0: 137
   Record con net_weight < 0: 0

✅ Dopo pulizia net_weight: 122385 record (rimossi 205)
   Percentuale rimossa: 0.2%


## 4. Advanced Feature Engineering (Tuo Progetto)

In [17]:
if 'batch_id' in data_clean.columns:
    # Analizza batch_id
    print(f"\n📊 Analisi batch_id:")
    print(f"   Valori totali: {len(data_clean)}")
    print(f"   Valori mancanti: {data_clean['batch_id'].isna().sum()} ({data_clean['batch_id'].isna().sum()/len(data_clean)*100:.1f}%)")
    print(f"   Valori unici: {data_clean['batch_id'].nunique()}")
    
    # OPZIONE 1: Crea feature "has_batch" (indica se il record ha un batch_id)
    data_clean['has_batch_id'] = (~data_clean['batch_id'].isna()).astype(int)
    
    # OPZIONE 2: Frequency encoding - quante volte appare questo batch_id?
    # Utile per capire se è un batch grande o piccolo
    batch_counts = data_clean['batch_id'].value_counts().to_dict()
    data_clean['batch_frequency'] = data_clean['batch_id'].map(batch_counts).fillna(0)
    
    # OPZIONE 3: Statistiche aggregate per batch (peso medio del batch)
    batch_stats = data_clean.groupby('batch_id')['net_weight'].agg(['mean', 'std', 'count']).reset_index()
    batch_stats.columns = ['batch_id', 'batch_mean_weight', 'batch_std_weight', 'batch_count']
    data_clean = data_clean.merge(batch_stats, on='batch_id', how='left')
    
    # Riempi i missing per i record senza batch_id
    data_clean['batch_mean_weight'] = data_clean['batch_mean_weight'].fillna(data_clean['net_weight'].mean())
    data_clean['batch_std_weight'] = data_clean['batch_std_weight'].fillna(0)
    data_clean['batch_count'] = data_clean['batch_count'].fillna(1)
    
    # Ora puoi eliminare batch_id originale (abbiamo estratto l'informazione utile)
    data_clean = data_clean.drop('batch_id', axis=1)
    
    print(f"\n✅ batch_id trasformato in feature:")
    print(f"   - has_batch_id: flag presenza batch")
    print(f"   - batch_frequency: frequenza del batch")
    print(f"   - batch_mean_weight: peso medio del batch")
    print(f"   - batch_std_weight: variabilità del batch")
    print(f"   - batch_count: dimensione del batch")
    print(f"\n✅ Colonna originale 'batch_id' rimossa (feature estratte)")
else:
    print("⚠️ Colonna 'batch_id' non trovata")


📊 Analisi batch_id:
   Valori totali: 122385
   Valori mancanti: 57744 (47.2%)
   Valori unici: 64641

✅ batch_id trasformato in feature:
   - has_batch_id: flag presenza batch
   - batch_frequency: frequenza del batch
   - batch_mean_weight: peso medio del batch
   - batch_std_weight: variabilità del batch
   - batch_count: dimensione del batch

✅ Colonna originale 'batch_id' rimossa (feature estratte)


## 5. Purchase Order Features

In [18]:
# Feature: Ritardo di consegna (differenza tra data arrivo e data prevista)
data_clean['delivery_delay_days'] = (data_clean['date_arrival'] - data_clean['delivery_date']).dt.days
data_clean['is_late_delivery'] = (data_clean['delivery_delay_days'] > 0).astype(int)
data_clean['is_early_delivery'] = (data_clean['delivery_delay_days'] < 0).astype(int)

# Feature: Differenza tra peso ricevuto e quantità ordinata
data_clean['weight_vs_quantity_ratio'] = data_clean['net_weight'] / data_clean['quantity'].replace(0, np.nan)
data_clean['weight_quantity_diff'] = data_clean['net_weight'] - data_clean['quantity']

# Feature: Tempo tra creazione ordine e arrivo
data_clean['order_to_arrival_days'] = (data_clean['date_arrival'] - data_clean['created_date_time']).dt.days

print(f"\n✅ Nuove feature create da purchase_orders:")
print(f"   - delivery_delay_days: ritardo/anticipo consegna")
print(f"   - is_late_delivery: flag consegna in ritardo")
print(f"   - is_early_delivery: flag consegna anticipata")
print(f"   - weight_vs_quantity_ratio: rapporto peso/quantità")
print(f"   - weight_quantity_diff: differenza peso-quantità")
print(f"   - order_to_arrival_days: giorni da ordine ad arrivo")

# Mostra statistiche
print(f"\n📊 Statistiche ritardi:")
print(f"   Consegne in ritardo: {data_clean['is_late_delivery'].sum()} ({data_clean['is_late_delivery'].mean()*100:.1f}%)")
print(f"   Consegne anticipate: {data_clean['is_early_delivery'].sum()} ({data_clean['is_early_delivery'].mean()*100:.1f}%)")
print(f"   Ritardo medio: {data_clean['delivery_delay_days'].mean():.1f} giorni")
print(f"   Ritardo mediano: {data_clean['delivery_delay_days'].median():.1f} giorni")


✅ Nuove feature create da purchase_orders:
   - delivery_delay_days: ritardo/anticipo consegna
   - is_late_delivery: flag consegna in ritardo
   - is_early_delivery: flag consegna anticipata
   - weight_vs_quantity_ratio: rapporto peso/quantità
   - weight_quantity_diff: differenza peso-quantità
   - order_to_arrival_days: giorni da ordine ad arrivo

📊 Statistiche ritardi:
   Consegne in ritardo: 9855 (8.1%)
   Consegne anticipate: 108839 (88.9%)
   Ritardo medio: -35.6 giorni
   Ritardo mediano: -15.0 giorni


## 6. Missing Values Handling

In [19]:
print("\n📊 Missing values dopo merge e nuove feature:")
missing_summary = data_clean.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
print(missing_summary)

# Strategia 1: IDs - Imputa con -1 (indica assenza/non trovato)
data_clean['rm_id'] = data_clean['rm_id'].fillna(-1)
data_clean['product_id'] = data_clean['product_id'].fillna(-1)
data_clean['purchase_order_id'] = data_clean['purchase_order_id'].fillna(-1)
data_clean['purchase_order_item_no'] = data_clean['purchase_order_item_no'].fillna(-1)

# Strategia 2: Feature numeriche da PO - Imputa con 0 (indica nessun match)
# Questo è più informativo di MICE perché preserva il significato strutturale
numeric_features = ['delivery_delay_days', 'weight_vs_quantity_ratio', 'weight_quantity_diff', 
                    'order_to_arrival_days', 'quantity']
for col in numeric_features:
    if col in data_clean.columns:
        data_clean[col] = data_clean[col].fillna(0)

# Strategia 3: Categorici - Imputa con "Unknown"
if 'status' in data_clean.columns:
    data_clean['status'] = data_clean['status'].fillna('Unknown')

print(f"\n✅ Missing values gestiti con strategie domain-specific")
print(f"   (Feature engineering > MICE per questo tipo di missing)")
print(f"📊 Dataset finale pronto per feature engineering: {len(data_clean)} record")
print(f"📊 Numero di colonne: {len(data_clean.columns)}")
print(f"\n📋 Colonne finali:")
print(data_clean.columns.tolist())

data_clean.head(10)


📊 Missing values dopo merge e nuove feature:
rm_id                       2
product_id                  2
purchase_order_id           2
purchase_order_item_no      2
quantity                    2
delivery_date               2
created_date_time           2
status                      2
product_version             2
delivery_delay_days         2
weight_vs_quantity_ratio    2
weight_quantity_diff        2
order_to_arrival_days       2
dtype: int64

✅ Missing values gestiti con strategie domain-specific
   (Feature engineering > MICE per questo tipo di missing)
📊 Dataset finale pronto per feature engineering: 122385 record
📊 Numero di colonne: 25

📋 Colonne finali:
['rm_id', 'product_id', 'purchase_order_id', 'purchase_order_item_no', 'receival_item_no', 'date_arrival', 'receival_status', 'net_weight', 'supplier_id', 'quantity', 'delivery_date', 'created_date_time', 'status', 'product_version', 'has_batch_id', 'batch_frequency', 'batch_mean_weight', 'batch_std_weight', 'batch_count', 'deli

,rm_id,product_id,purchase_order_id,purchase_order_item_no,receival_item_no,date_arrival,receival_status,net_weight,supplier_id,quantity,...,batch_frequency,batch_mean_weight,batch_std_weight,batch_count,delivery_delay_days,is_late_delivery,is_early_delivery,weight_vs_quantity_ratio,weight_quantity_diff,order_to_arrival_days
0,365.0,91900143.0,208545.0,10.0,1,2004-06-15 11:34:00+00:00,Completed,11420.0,52062,1975000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.005782,-1963580.0,153.0
1,365.0,91900143.0,208545.0,10.0,2,2004-06-15 11:34:00+00:00,Completed,13760.0,52062,1975000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.006967,-1961240.0,153.0
2,365.0,91900143.0,208490.0,10.0,1,2004-06-15 11:38:00+00:00,Completed,11281.0,50468,1500000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.007521,-1488719.0,158.0
3,365.0,91900143.0,208490.0,10.0,2,2004-06-15 11:38:00+00:00,Completed,13083.0,50468,1500000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.008722,-1486917.0,158.0
4,379.0,91900296.0,210435.0,20.0,1,2004-06-15 11:40:00+00:00,Completed,23910.0,52577,125000.0,...,0.0,12987.088271,0.0,1.0,-15.0,0,1,0.191280,-101090.0,-10.0
5,389.0,91900330.0,208535.0,30.0,1,2004-06-15 11:43:00+00:00,Completed,8680.0,55251,350000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.024800,-341320.0,153.0
6,365.0,91900143.0,208532.0,10.0,1,2004-06-15 11:46:00+00:00,Completed,14840.0,20023,3500000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.004240,-3485160.0,153.0
7,369.0,91900146.0,208532.0,30.0,2,2004-06-15 11:46:00+00:00,Completed,6745.0,20023,600000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.011242,-593255.0,153.0
8,366.0,91900160.0,208532.0,20.0,3,2004-06-15 11:46:00+00:00,Completed,3015.0,20023,600000.0,...,0.0,12987.088271,0.0,1.0,-199.0,0,1,0.005025,-596985.0,153.0
9,365.0,91900143.0,208537.0,10.0,1,2004-06-16 06:26:00+00:00,Completed,25060.0,50387,6575000.0,...,0.0,12987.088271,0.0,1.0,-198.0,0,1,0.003811,-6549940.0,154.0


In [20]:
# Salva dataset intermedio (record-level con tutte le feature)
data_clean.to_csv('data_clean_detailed.csv', index=False)
print(f"✅ Dataset dettagliato salvato: {data_clean.shape}")
print("   Prossimo step: Aggregazione giornaliera per repository reference approach")

✅ Dataset dettagliato salvato: (122385, 25)
   Prossimo step: Aggregazione giornaliera per repository reference approach


## 7. Daily Aggregation (Repository Reference Approach)

In [21]:
# ✅ AGGREGAZIONE GIORNALIERA: Repository Reference Approach
# Aggrega per (date_arrival, rm_id) come nel repository di riferimento

# Converti date_arrival a solo data (rimuovi time component)
data_clean['date_only'] = data_clean['date_arrival'].dt.date
data_clean['date_only'] = pd.to_datetime(data_clean['date_only'])

print(f"📊 Prima dell'aggregazione: {len(data_clean)} record")

# Aggrega per (data, rm_id) - core del repository di riferimento
daily_receivals = data_clean.groupby(['date_only', 'rm_id'], as_index=False).agg({
    'net_weight': 'sum',  # TARGET: Somma peso giornaliero per materiale
    'purchase_order_id': 'count',  # Numero di delivery events
    'supplier_id': 'first',  # Info supplier
    'product_id': 'first',  # Info prodotto
    'quantity': 'sum',  # Somma quantità totale giornaliera
    'delivery_delay_days': 'mean',  # Media ritardo giornaliero
    'receival_status': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0]  # Status più frequente
}).rename(columns={
    'date_only': 'date_arrival',
    'purchase_order_id': 'num_deliveries'
})

print(f"✅ DOPO aggregazione: {len(daily_receivals)} record")
print(f"📊 Riduzione: {len(data_clean)} → {len(daily_receivals)} ({len(daily_receivals)/len(data_clean)*100:.1f}%)")
print(f"📊 Materiali unici: {daily_receivals['rm_id'].nunique()}")
print(f"📊 Range date: {daily_receivals['date_arrival'].min()} → {daily_receivals['date_arrival'].max()}")

# Analisi target dopo aggregazione (importante per Quantile Loss)
print(f"\n📈 Target Analysis (net_weight) dopo aggregazione:")
print(f"   Media: {daily_receivals['net_weight'].mean():.0f} kg")
print(f"   Mediana: {daily_receivals['net_weight'].median():.0f} kg")
print(f"   Std: {daily_receivals['net_weight'].std():.0f} kg")
print(f"   Zero values: {(daily_receivals['net_weight'] == 0).sum()} ({(daily_receivals['net_weight'] == 0).mean()*100:.1f}%)")

# Salva dataset finale per feature engineering
daily_receivals.to_csv('data_clean.csv', index=False)
print(f"\n✅ Dataset finale salvato: 'data_clean.csv' ({daily_receivals.shape})")
print("🎯 PRONTO per feature engineering avanzato!")

daily_receivals.head()

📊 Prima dell'aggregazione: 122385 record
✅ DOPO aggregazione: 41907 record
📊 Riduzione: 122385 → 41907 (34.2%)
📊 Materiali unici: 204
📊 Range date: 2004-06-15 00:00:00 → 2024-12-19 00:00:00

📈 Target Analysis (net_weight) dopo aggregazione:
   Media: 37927 kg
   Mediana: 22998 kg
   Std: 48486 kg
   Zero values: 0 (0.0%)

✅ Dataset finale salvato: 'data_clean.csv' ((41907, 9))
🎯 PRONTO per feature engineering avanzato!
✅ DOPO aggregazione: 41907 record
📊 Riduzione: 122385 → 41907 (34.2%)
📊 Materiali unici: 204
📊 Range date: 2004-06-15 00:00:00 → 2024-12-19 00:00:00

📈 Target Analysis (net_weight) dopo aggregazione:
   Media: 37927 kg
   Mediana: 22998 kg
   Std: 48486 kg
   Zero values: 0 (0.0%)

✅ Dataset finale salvato: 'data_clean.csv' ((41907, 9))
🎯 PRONTO per feature engineering avanzato!


,date_arrival,rm_id,net_weight,num_deliveries,supplier_id,product_id,quantity,delivery_delay_days,receival_status
0,2004-06-15,365.0,83784.0,7,52062,91900143.0,16400000.0,-199.0,Completed
1,2004-06-15,366.0,3015.0,1,20023,91900160.0,600000.0,-199.0,Completed
2,2004-06-15,368.0,13500.0,1,57292,91900170.0,75000.0,-15.0,Completed
3,2004-06-15,369.0,6745.0,1,20023,91900146.0,600000.0,-199.0,Completed
4,2004-06-15,379.0,23910.0,1,52577,91900296.0,125000.0,-15.0,Completed


## ✅ Data Cleaning Completato - Summary

In [22]:
print("="*80)
print("✅ DATA CLEANING COMPLETATO - VERSIONE INTEGRATA")
print("="*80)

print(f"""
🎯 APPROCCIO UNIFICATO IMPLEMENTATO:

1. 📊 TUO PROGETTO (Advanced Features):
   ✅ Batch ID feature engineering 
   ✅ Purchase Order features (ritardi, ratio peso/quantità)
   ✅ Missing values gestiti con domain knowledge
   ✅ Feature da temporal data

2. 📈 REPOSITORY RIFERIMENTO (Data Structure):
   ✅ Aggregazione giornaliera (date_arrival, rm_id)
   ✅ Riduzione dataset da 122k a 42k record (-66%)
   ✅ Target preparation per Quantile Loss
   ✅ Time-series ready structure

📊 OUTPUT FINALE:
   File: data_clean.csv
   Shape: {daily_receivals.shape}
   Columns: {list(daily_receivals.columns)}
   Target: net_weight (media {daily_receivals['net_weight'].mean():.0f} kg)
   
🚀 PROSSIMO STEP: feature_engineering.ipynb
   Input: data_clean.csv 
   Processo: 48+ advanced features per ML
""")

print("="*80)

✅ DATA CLEANING COMPLETATO - VERSIONE INTEGRATA

🎯 APPROCCIO UNIFICATO IMPLEMENTATO:

1. 📊 TUO PROGETTO (Advanced Features):
   ✅ Batch ID feature engineering 
   ✅ Purchase Order features (ritardi, ratio peso/quantità)
   ✅ Missing values gestiti con domain knowledge
   ✅ Feature da temporal data

2. 📈 REPOSITORY RIFERIMENTO (Data Structure):
   ✅ Aggregazione giornaliera (date_arrival, rm_id)
   ✅ Riduzione dataset da 122k a 42k record (-66%)
   ✅ Target preparation per Quantile Loss
   ✅ Time-series ready structure

📊 OUTPUT FINALE:
   File: data_clean.csv
   Shape: (41907, 9)
   Columns: ['date_arrival', 'rm_id', 'net_weight', 'num_deliveries', 'supplier_id', 'product_id', 'quantity', 'delivery_delay_days', 'receival_status']
   Target: net_weight (media 37927 kg)

🚀 PROSSIMO STEP: feature_engineering.ipynb
   Input: data_clean.csv 
   Processo: 48+ advanced features per ML

